# Week 7

## Objectives so far
* Code a graph generator for Unity
* Code force fields in Unity
* Code Lennard-Jones forces for polymer chain entanglement

## July 25

Today I should be working on the implementation of force fields in Unity. Among the lots of things I found while figuring out how to do this, I came across this particular paper which could be extremely useful as it works with molecules containing glucose modules:

>**Title:** Martini Coarse-Grained Force Field: Extension to Carbohydrates
>
>**Author(s):** Cesar A. López, Andrzej J. Rzepiela, Alex H. de Vries, Lubbert Dijkhuizen, Philippe H. Hünenberger, and Siewert J. Marrink
>
>**Link:** [PDF Article](https://pubs.acs.org/doi/pdf/10.1021/ct900313w)
>
>**Important Points:**
>* They describe a couple of changes made to the original MARTINI 2.0 model for adapting it to carbohydrates.
>* It is important to note how they describe the fact that the parametrization of the bonded and non-bonded interaction equations are largely dependent on the results of AA simulations (i.e. they draw a trajectory, and try to match the parameters so that we obtain values that are closer to those of the AA simulations).

---

### Force-Fields in Unity through CG models

Right now I have a small plan for this. For the implementation of these Force Fields, I have considered the use of at least 4 scripts:

* One for the characteristics of the Agent Bead (e.g. Apolar, donor and 4 in polarization scale)
* Another one for the bonded potential functions; which must then be translated into physics
* Another one for the non-bonded potential functions; which must then be translated into physics.
* RL functionality

Let's start with the CG characteristics of the Agent bead. The idea is to have two characteristics to each bead.
```c#
string beadType;        // Whether the bead is Charged (Q), Apolar (A), Polar (P), or Semipolar (N)
string beadSubType;     // The level of polarity of each bead (it is from low 1 to high 5) 
                        // or whether the bead is a donor (d), acceptor (a), both (da), or none (0)
```

The idea in containing this information is to be able to make a dictionary in C# for the parameters of the non-bonded interactions (given by the Lennard-Jones equation):

```c#
public string determineIntLevel (Dictionary <string, string> beadOne, Dictionary <string, string> beadTwo) {
    Dictionary<string[], 
}

Dictionary<string, float> interactionLevels = new Dictionary<string, float>() {
    {"O", 5.6}, {"I", 5}, {"II", 4.5}, {"III", 4}, {"IV", 3.5},
    {"V", 3.1}, {"VI", 2.7}, {"VII", 2.3}, {"VIII", 2.0}, {"IX", 2.0}
};

// The ints are made so that the scale is in [ε] = kJ/mol
```

Let's suppose this bead data is contained in some file named `beadData.cs`. Then, we can write the script for the non-bonded interactions:

```c#
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

// Suppose that we have a list of all the nearby beads called neighbours.
// Each Bead object neighbour will have the methods neighbor.types for an array [Type, SubType].
// Assume that calling gameObject refers to the current Agent bead this script is attached to.
// effectiveSize() would determine the value for sigma
// distance() would compute the distance between a bead and its neighbor

internal class nonBondedForces : MonoBehaviour {
    Beads[] neighbours = ...;
    
    for (int i = 0; i < neighbours.Length; i++) {
        Bead neighbour = neighbours[i];
    
        float epsilon = interactionLevels[determineIntLevel(gameObject.types, neighbor.types)];
        float sigma = effectiveSize(gameObject.types, neighbor.types);
        float distance = distance(gameObject, neighbor);
    
        U_LJ = 4 * epsilon * ((sigma/distance)**12 - (sigma/distance)**6);
        Force = -derivative(U_LJ);
    }
}
```

## July 26

Today I keep doing the script for the forces in Unity. Also, I will have to have a meeting in order to arrange how will the presentation for Friday be organized.

### Meeting (7/26/2022 - 1:30pm)

The meeting was quite fast. We agreed on me presenting about Molecular Dynamics and the use of the MARTINI model to represent force fields. In addition to that, it was noted that the lightning talks would have to include some sort of Machine Learning implementation/explanation, for which I agreed to study the wait in which force fields get implemented or altered in the RL model and include that as part of my presentation.

## July 27
During this day, I tried looking more into tutorials of Unity to understand using the components and the manipulation of scripts in the Unity project.

As a result, I managed to make some sketches of how the end result of the algorithm could look like.

## July 28
Today, I finally managed to have somewhat of a code representation for the Non-Bonded and Bonded Interactions from the MARTINI model. Let's start by parts:

Firstly, we have `EmbeddedBead.cs`. It doesn't have anything particularly stricking; it just contains two values: one for the type of bead and another one for the sub-type of bead:

```c#
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class EmbeddedBead : MonoBehaviour
{
    // The type will be assigned to be either P (Polar), N (Semipolar), A (Apolar), and Q (Charged) and BP (only for antifreeze particles)
    public string Type;

    // The type will be assigned to be one of two:
    //       * Hydrogen-bonding: a (acceptor), d (donor), da (both), 0 (none)
    //       * Polarity Level: 1 (lowest), 2, 3, 4, 5 (highest)
    public string SubType;

    // Start is called before the first frame update
    void Start()
    {
        
    }

    // Update is called once per frame
    void Update()
    {
        
    }
}
```

Then, using this as a reference for the forces, we go onto `NonBondedForce.cs`, which would implement the non-bonding interactions from the MARTINI model itself:

```c#
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using System;

public class NonBondedForce : MonoBehaviour
{
    public List<GameObject> neighbours;
    public List<float> potentials;
    public List<float> distances;

    private double interactionLevel(GameObject one, GameObject two) {
        // We get the embedded info
        EmbeddedBead beadOne = one.GetComponent<EmbeddedBead>();
        EmbeddedBead beadTwo = two.GetComponent<EmbeddedBead>();

        // We get the types and subtypes
        List<string> oneTypes = new List<string> {beadOne.Type, beadOne.SubType};
        List<string> twoTypes = new List<string> {beadTwo.Type, beadTwo.SubType};

        // We compute the Interaction Matrix
        Dictionary<List<string>, int> typeMappings = new Dictionary<List<string>, int>() {
            {new List<string>{"Q", "da"}, 0}, {new List<string>{"Q", "d"}, 1}, {new List<string>{"Q", "a"}, 2}, {new List<string>{"Q", "0"}, 3},
            {new List<string>{"P", "5"}, 4}, {new List<string>{"P", "4"}, 5}, {new List<string>{"P", "3"}, 6}, {new List<string>{"P", "2"}, 7}, {new List<string>{"P", "1"}, 8},
            {new List<string>{"N", "da"}, 9}, {new List<string>{"N", "d"}, 10}, {new List<string>{"N", "a"}, 11}, {new List<string>{"N", "0"}, 12},
            {new List<string>{"C", "5"}, 13}, {new List<string>{"C", "4"}, 14}, {new List<string>{"C", "3"}, 15}, {new List<string>{"C", "2"}, 16}, {new List<string>{"C", "1"}, 17},
            {new List<string>{"BP", "4"}, 18}
        };

        List<List<string>> levels = new List<List<string>>();
        levels.Add(new List<string> {"O", "O", "O", "II", "O", "O", "O", "I", "I", "I", "I", "I", "IV", "V", "VI", "VII", "IX", "IX", "O"});
        levels.Add(new List<string> {"O", "I", "O", "II", "O", "O", "O", "I", "I", "I", "III", "I", "IV", "V", "VI", "VII", "IX", "IX", "O"});
        levels.Add(new List<string> {"O", "O", "I", "II", "O", "O", "O", "I", "I", "I", "I", "III", "IV", "V", "VI", "VII", "IX", "IX", "O"});
        levels.Add(new List<string> {"II", "II", "II", "IV", "I", "O", "I", "II", "III", "III", "III", "III", "IV", "V", "VI", "VII", "IX", "IX", "O"});
        levels.Add(new List<string> {"O", "O", "O", "I", "O", "O", "O", "O", "O", "I", "I", "I", "IV", "V", "VI", "VI", "VII", "VIII", "O"});
        levels.Add(new List<string> {"O", "O", "O", "O", "O", "I", "I", "II", "II", "III", "III", "III", "IV", "V", "VI", "VI", "VII", "VIII", "O"});
        levels.Add(new List<string> {"O", "O", "O", "I", "O", "I", "I", "II", "II", "II", "II", "II", "IV", "IV", "V", "V", "VI", "VII", "O"});
        levels.Add(new List<string> {"I", "I", "I", "II", "O", "II", "II", "II", "II", "II", "II", "II", "III", "IV", "IV", "V", "VI", "VII", "O"});
        levels.Add(new List<string> {"I", "I", "I", "III", "O", "II", "II", "II", "II", "II", "II", "II", "III", "IV", "IV", "IV", "V", "VI", "O"});
        levels.Add(new List<string> {"I", "I", "I", "III", "I", "III", "II", "II", "II", "II", "II", "II", "IV", "IV", "V", "VI", "VI", "VI", "O"});
        levels.Add(new List<string> {"I", "III", "I", "III", "I", "III", "II", "II", "II", "II", "III", "II", "IV", "IV", "V", "VI", "VI", "VI", "O"});
        levels.Add(new List<string> {"I", "I", "III", "III", "I", "III", "II", "II", "II", "II", "II", "III", "IV", "IV", "V", "VI", "VI", "VI", "O"});
        levels.Add(new List<string> {"IV", "IV", "IV", "IV", "IV", "IV", "IV", "III", "III", "IV", "IV", "IV", "IV", "IV", "IV", "IV", "V", "VI", "O"});
        levels.Add(new List<string> {"V", "V", "V", "V", "V", "V", "IV", "IV", "IV", "IV", "IV", "IV", "IV", "IV", "IV", "IV", "V", "V", "O"});
        levels.Add(new List<string> {"VI", "VI", "VI", "VI", "VI", "VI", "V", "IV", "IV", "V", "V", "V", "IV", "IV", "IV", "IV", "V", "V", "O"});
        levels.Add(new List<string> {"VII", "VII", "VII", "VII", "VI", "VI", "V", "V", "IV", "VI", "VI", "VI", "IV", "IV", "IV", "IV", "IV", "IV", "O"});
        levels.Add(new List<string> {"IX", "IX", "IX", "IX", "VII", "VII", "VI", "VI", "V", "VI", "VI", "VI", "V", "V", "V", "IV", "IV", "IV", "O"});
        levels.Add(new List<string> {"IX", "IX", "IX", "IX", "VIII", "VIII", "VII", "VII", "VI", "VI", "VI", "VI", "VI", "V", "V", "IV", "IV", "IV", "O"});
        levels.Add(new List<string> {"O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"});

        // Interaction Levels with their numeric values assigned to them in a scale [ε] = kJ/mol
        Dictionary<string, double> interactionLevels = new Dictionary<string, double>() {
            {"O", 5.6}, {"I", 5}, {"II", 4.5}, {"III", 4}, {"IV", 3.5},
            {"V", 3.1}, {"VI", 2.7}, {"VII", 2.3}, {"VIII", 2.0}, {"IX", 2.0}
        };

        string level = levels[typeMappings[oneTypes]][typeMappings[twoTypes]];
        return interactionLevels[level];
    }

    private double effectiveSize(GameObject one, GameObject two) {
        // We get the embedded info
        EmbeddedBead beadOne = one.GetComponent<EmbeddedBead>();
        EmbeddedBead beadTwo = two.GetComponent<EmbeddedBead>();

        // We get the types and subtypes
        List<string> oneTypes = new List<string> {beadOne.Type, beadOne.SubType};
        List<string> twoTypes = new List<string> {beadTwo.Type, beadTwo.SubType};

        if (oneTypes[0] == "Q" && twoTypes[0] == "Q") {
            return (0.62 * Math.Pow(10, -9));
        }
        else if ( ((oneTypes == new List<string> {"C", "1"}) || (oneTypes == new List<string> {"C", "2"})) 
        && ((twoTypes == new List<string> {"C", "1"}) || (twoTypes == new List<string> {"C", "2"})) ){
            return (0.62 * Math.Pow(10,-9));
        }
        else if ((oneTypes == new List<string> {"BP", "4"}) || (twoTypes == new List<string> {"BP", "4"})) {
            return (0.57 * Math.Pow(10,-9));
        }
        else {
            return (0.47 * Math.Pow(10,-9));
        }
    }

    // Start is called before the first frame update
    void Start()
    {
        List<GameObject> neighbours = new List<GameObject>();
        // I have to revise how to scale the value of the radius to a real world bead
        // Each bead, technically, has a radius of at most σ = 1.2nm
        Collider[] others = Physics.OverlapSphere(transform.position, 1.2f * (float) Math.Pow(10, -9));  

        // Takes from all the nearby colliders, only the ones that are beads
        // and then stores them in a list of GameObjects
        foreach (var other in others) {
            if (other.gameObject.tag == "Bead") {
                neighbours.Add(other.gameObject);
            }
        }

        // Initialize the register of potentials and distances to all 0s
        List<float> potentials = new List<float>(new float[neighbours.Count]);
        List<float> distances = new List<float>(new float[neighbours.Count]);
    }

    // Update is called once per frame
    void Update()
    {
        Debug.Log(neighbours);
        for (int i = 0; i < neighbours.Count; i++) {
            GameObject current = this.gameObject;
            GameObject neighbour = neighbours[i];
        
            float epsilon = (float) interactionLevel(current, neighbour);
            float sigma = (float) effectiveSize(current, neighbour);
            float distance = Vector3.Distance(transform.position, neighbour.transform.position);
        
            float U_LJ = 4 * epsilon * ((float) Math.Pow((sigma / distance), 12) - (float) Math.Pow((sigma / distance), 6));

            double F_nb = (potentials[i] - U_LJ)/(distance - distances[i]);

            distances[i] = distance;
            potentials[i] = U_LJ;

            Vector3 direction_from_one = (neighbour.transform.position - transform.position);
            Vector3 direction_from_two = (transform.position - neighbour.transform.position);
            Rigidbody one_Rigidbody = current.GetComponent<Rigidbody>();
            Rigidbody two_Rigidbody = neighbour.GetComponent<Rigidbody>();
            one_Rigidbody.AddForce(direction_from_one * (float) F_nb);
            two_Rigidbody.AddForce(direction_from_two * (float) F_nb * -1f);
        }
    }
}
```

And finally, we have a sketch of the Bonded Interactions, which will likely have to be updated as soon as we get a valid representation for our bead/coarse-grained model:

```c#
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using System;

public class BondedForce : MonoBehaviour
{
    // TODO: We initialize the parameters from another script
    public double K_bond;
    public double R_bond;
    public double K_angle;
    public double theta_0;
    public double K_pd;
    public double phi_pd;

    // We will have to obtain the parameters for the bead somehow
    // private embeddedParameters embedded;
    // embeddedParameters embedded = GameObject.GetComponent<embeddedParameters>();
    // 
    // From here we will have:
    // embeddedParameters.R
    // embeddedParameters.theta
    // embeddedParameters.phi

    static public List<GameObject> bondedNeighbours;
    public List<float> potentials;
    public List<float> distances;

    // Start is called before the first frame update
    void Start()
    {
        // Looks for a way to search for all bonded beads to a local bead to then iterate around it
        HingeJoint[] joints = GetComponents<HingeJoint>();

        for (int i = 0; i < joints.Length; i++) {
            if (joints[i].connectedBody) {
                bondedNeighbours.Add(joints[i].gameObject);
            }
        }

        List<float> potentials = new List<float>(new float[bondedNeighbours.Count]);
        List<float> distances = new List<float>(new float[bondedNeighbours.Count]);
    }

    // Update is called once per frame
    void Update()
    {
        for (int i = 0; i < bondedNeighbours.Count; i++) {
            GameObject current = this.gameObject;
            GameObject neighbour = bondedNeighbours[i];
            public double V_bond = (0.5) * K_bond * (embeddedParameters.R - R_bond);
            public double V_angle = (0.5) * K_angle * (cos(embeddedParameters.theta) - cos(theta_0))**2;
            public double V_pd = K_pd * (1 + cos(embeddedParameters.phi - phi_pd));

            float distance = Vector3.Distance(transform.position, neighbour.transform.position);
            // And from there, calculate and implement a force
            double F_b = -(V_bond + V_angle + V_pd - potentials[i])/(distance - distances[i]);

            distances[i] = distance;
            potentials[i] = (V_bond + V_angle + V_pd);

            Vector3 direction_from_one = (neighbour.transform.position - transform.position);
            Vector3 direction_from_two = (transform.position - neighbour.transform.position);
            Rigidbody one_Rigidbody = current.GetComponent<Rigidbody>();
            Rigidbody two_Rigidbody = neighbour.GetComponent<Rigidbody>();
            one_Rigidbody.AddForce(direction_from_one * (float) F_b);
            two_Rigidbody.AddForce(direction_from_two * (float) F_b * -1f);   
        }
    }
}
```

### Meeting (3:30pm - 7/28/2022)

>**Elena Garza**
>
>* Elena updated the Thermoplastic Starch document. She went through the blends of thermoplastic starch. In addition, she made a comparison between cellulose acetate and thermoplastic starch in regards to which one is better for biodegradability.
>* She showed us the slides for her presentation this week's Friday.

>**Jehan Ahmed**
>
>* For thermoplastic starch, she could not find much information in general.
>* Fractionation for cellulose acetate. The first paper was not very useful because it went into the technique in-depth. The second was useful, however; it contained information regarding the degree of polymerization. She looked more into the degree of polymerization, trying to answer the question of how many monomers are in cellulose acetate.

For both, Paloma asked them how is the data compilation going. Jehan has around 30 tables from the book compilated in the Dropbox folder; also, she is getting close to the 10th chapter in the book.

>**Neil Malur**
>
>* He has built nodes and edges for the graph implementation. 
>* He has been changing from APIs due to issues in their development.

>**Andy Emmel**
>
>* He has been messing around the atomistic model with LJ forces implemented.
>* It looks good but he wants to observe if it can be made so that it is physically accurate.

Paloma spoke with IBM. They seem to be very interested, as it incorporates physics. A signed joint agreement has to be done to apply for funding and be able to work with them. She also spoke with a UCLA PhD candidate that works in the Materials Genome Project. They invented a biopolymer and are observing if it is synthesizable. They are available for collaboration. She will contact people from GeorgiaTech.

With IBM it is important to generate Open Source tools. That is part of the plan.

## June 29